In [26]:
import copy
import cv2 as cv
import math
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import pytesseract
import os

In [27]:
def show_wait_destroy(winname, img):
    cv.imshow(winname, img)
    cv.moveWindow(winname, 500, 0)
    cv.waitKey(0)
    cv.destroyWindow(winname)

In [28]:
image = cv.imread("imgs/crop.jpg",cv.IMREAD_COLOR)
show_wait_destroy("image",image)
gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
show_wait_destroy("gray", gray)
gray = cv.bitwise_not(gray)
bw = cv.adaptiveThreshold(gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 15, -2)
show_wait_destroy("binary", bw)
# enlarge = cv2.resize(image_copy,None,fx=2.0, fy=2.0, interpolation = cv2.INTER_CUBIC)
# cv2.imshow("image",enlarge)
# cv2.waitKey(0)
# denoiced_image = cv2.fastNlMeansDenoisingColored(image_copy,None,10,10,7,21)
# cv2.imshow("denoiced_image",denoiced_image)
# cv2.waitKey(0)


In [29]:
horizontal = np.copy(bw)
vertical = np.copy(bw)
cols = horizontal.shape[1]
horizontal_size = int(cols / 30)
horizontalStructure = cv.getStructuringElement(cv.MORPH_RECT, (horizontal_size, 1))
# Apply morphology operations
horizontal = cv.erode(horizontal, horizontalStructure)
horizontal = cv.dilate(horizontal, horizontalStructure)

# Show extracted horizontal lines
show_wait_destroy("horizontal", horizontal)

In [30]:
rows = vertical.shape[0]
verticalsize = int(rows / 5)
# Create structure element for extracting vertical lines through morphology operations
verticalStructure = cv.getStructuringElement(cv.MORPH_RECT, (1, verticalsize))

# Apply morphology operations
vertical = cv.erode(vertical, verticalStructure)
vertical = cv.dilate(vertical, verticalStructure)

# Show extracted vertical lines
show_wait_destroy("vertical", vertical)

In [23]:
# Inverse vertical image
vertical = cv.bitwise_not(vertical)
show_wait_destroy("vertical_bit", vertical)

In [32]:
joints = cv.bitwise_and(horizontal,vertical)
show_wait_destroy("Points of Intersection", joints)

In [54]:
# edges = cv.adaptiveThreshold(vertical, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 3, -2)
# show_wait_destroy("edges", edges)


# kernel = np.ones((2, 2), np.uint8)
# edges = cv.dilate(edges, kernel)
# show_wait_destroy("dilate", edges)

# smooth = np.copy(vertical)

# smooth = cv.blur(smooth, (2, 2))

# (rows, cols) = np.where(edges != 0)
# vertical[rows, cols] = smooth[rows, cols]

# show_wait_destroy("smooth - final", vertical)